In [1]:
#データの読み込み
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/chapter09/train.txt', sep="\t")
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/chapter09/test.txt', sep="\t")
valid = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/chapter09/valid.txt', sep="\t")
# データ数の確認
print('学習データ')
print(train['CATEGORY'].value_counts())
print('検証データ')
print(valid['CATEGORY'].value_counts())
print('評価データ')
print(test['CATEGORY'].value_counts())

学習データ
CATEGORY
b    4502
e    4223
t    1219
m     728
Name: count, dtype: int64
検証データ
CATEGORY
b    562
e    528
t    153
m     91
Name: count, dtype: int64
評価データ
CATEGORY
b    563
e    528
t    152
m     91
Name: count, dtype: int64


In [2]:
# 単語の辞書を作成
from collections import Counter
words = []
for text in train['TITLE']:                #訓練データから文章を1つずつ取り出す
    for word in text.rstrip().split():     #文章を単語に分解
        words.append(word)                 #単語をリストに追加
c = Counter(words)                         #単語の出現回数を数える
print(c.most_common(10))                   #頻度上位10単語
word2id = {}                               #単語IDの辞書
for i, cnt in enumerate(c.most_common()):  #頻度上位10単語分繰り返す
    if cnt[1] > 1:                         #出現回数が1より大きい単語のみ
        word2id[cnt[0]] = i + 1            #辞書に単語とIDを紐付ける
for i, cnt in enumerate(word2id.items()):  #辞書の中身を確認
    if i >= 10:                            #10単語だけ表示
        break                              #for文を抜ける
    print(cnt[0], cnt[1])                  #単語とIDを表示

[('to', 2151), ('...', 2031), ('in', 1415), ('as', 1027), ('on', 1025), ('UPDATE', 1000), ('-', 991), ('for', 969), ('of', 957), ('The', 859)]
to 1
... 2
in 3
as 4
on 5
UPDATE 6
- 7
for 8
of 9
The 10


In [3]:
# 単語のID化
def tokenizer(text):                                 #単語IDのリストを返す関数
    words = text.rstrip().split()                    #単語に分解
    return [word2id.get(word, 0) for word in words]  #単語のIDに変換

sample = train.at[0, 'TITLE']                        #学習データの1つ目の文章
print(sample)                                        #文章を表示
print(tokenizer(sample))                             #文章を単語IDに変換

Justin Bieber Under Investigation For Attempted Robbery At Dave & Buster's
[66, 79, 733, 2094, 21, 4933, 6674, 35, 1514, 86, 0]


In [4]:
# RNNの作成
# モデルの構築
import random
import torch
from torch import nn
import torch.utils.data as data
from torchinfo import summary
import numpy as np

# 乱数のシードを設定
# parserなどで指定
seed = 1234

random.seed(seed)                                   # Python標準ライブラリの乱数のシードを設定
np.random.seed(seed)                                # Numpy乱数のシードを設定
torch.manual_seed(seed)                             # PyTorch乱数のシードを設定
torch.cuda.manual_seed(seed)                        # PyTorchのCUDA乱数のシードを設定
torch.backends.cudnn.benchmark = False              # PyTorchのCUDNNのベンチマークを使用しない  (cudnn内の非決定的な処理の固定化)
torch.backends.cudnn.deterministic = True           # PyTorchのCUDNNの定着を使用

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32      # 乱数生成のシードの初期値を設定
    np.random.seed(worker_seed)                     # Numpy乱数のシードを設定
    random.seed(worker_seed)                        # Python標準ライブラリの乱数のシードを設定

g = torch.Generator()                               # PyTorch乱数のシードを設定
g.manual_seed(seed)                                 # 乱数生成器にシードを設定

class RNN(nn.Module):                                                                               # RNNクラスを定義
    def __init__(self, vocab_size, emb_size, padding_idx, hidden_size, output_size, num_layers=1):  # コンストラクタ
        super().__init__()                                                                          # 親クラスのコンストラクタを呼ぶ
        self.emb = nn.Embedding(vocab_size, emb_size, padding_idx=padding_idx)                      # 単語埋め込み層
        self.rnn = nn.LSTM(emb_size, hidden_size, batch_first=True)                                 # RNN層
        self.fc = nn.Linear(hidden_size, output_size)                                               # 全結合層

    def forward(self, x, h0=None):                                                                  # 順伝播処理
        x = self.emb(x)                                                                             # 単語埋め込み
        x, h = self.rnn(x, h0)                                                                      # RNN
        x = x[:, -1, :]                                                                             # 最後のステップのみを抽出
        logits = self.fc(x)                                                                         # 全結合層
        return logits                                                                               # 出力

# パラメータの設定
VOCAB_SIZE = len(set(word2id.values())) + 2  # 辞書のID数 + unknown + パディングID
EMB_SIZE = 300                               # 単語埋め込みベクトルのサイズ
PADDING_IDX = len(set(word2id.values())) + 1 # パディングID
OUTPUT_SIZE = 4                              # 出力サイズ
HIDDEN_SIZE = 50                             # 隠れ層のサイズ
NUM_LAYERS = 1                               # RNN層の数

# モデルの定義
model = RNN(VOCAB_SIZE, EMB_SIZE, PADDING_IDX, HIDDEN_SIZE, OUTPUT_SIZE, NUM_LAYERS)  # RNNクラスのインスタンスを作成
print(model)                                                                          # モデルの構造を確認

RNN(
  (emb): Embedding(10328, 300, padding_idx=10327)
  (rnn): LSTM(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=4, bias=True)
)


In [5]:
x = torch.tensor([tokenizer(sample)], dtype=torch.int64)                           # 文章を単語IDに変換
print(x)                                                                           # 文章をIDでを表示
print(x.size())                                                                    # 文章のサイズを確認
print(nn.Softmax(dim=-1)(model(x)))                                                # 出力を確認

tensor([[  66,   79,  733, 2094,   21, 4933, 6674,   35, 1514,   86,    0]])
torch.Size([1, 11])
tensor([[0.2148, 0.2503, 0.2325, 0.3023]], grad_fn=<SoftmaxBackward0>)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
pip install torchinfo